In [25]:
import os

import re
from collections import Counter
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Input, Dropout, GRU,Dense, SimpleRNN, TimeDistributed, Activation, RepeatVector,Bidirectional, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

## RNN
***
- 입력과 출력을 시퀀스 단위로 처리하는 시퀀스 모델.
- 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고 있다.
- RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀이라고 하며, 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행한다.


현재 시점 t에서의 은닉 상태값을 $ h_t $ 라 하였을때, 은닉측의 메모리 셀은 $ h_t $ 를 계산하기 위해서 총 두개의 가중치를 가진다.
하나는 입력층을 위한 가중치 $ W_x $, 다른 하나는 이전 시점 $ t-1 $ 의 은닉 상태값인 $ h_{t-1} $ 을 위한 가중치 $ W_h $ 이다.


- 은닉층 : $ h_t = tanh(W_x x_t + W_hh{t-1} + b) $
- 출력층 : $ y_t = f(W_yh_t + b) $
- 단 , $f$는 비선형 활성화 함수 중 하나

    - $x_t$ : 단어 벡터, $d$ : 단어 벡터의 차원, $D_h$: 은닉 상태의 크기
    - $x_t = (d * 1)$
    - $W_x = (D_h * d)$
    - $W_h = (D_h * D_h)$
    - $h_{t-1} = (D_h * 1)$
    - $b = (D_h * 1)$


#### RNN Input
***
- RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력 받는다. 
- batch_size = 한 번에 학습하는 데이터의 개수

- hidden_units = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통  __128, 256, 512, 1024__ 등의 값을 가진다.
- timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수. (__자연어 처리에서는 문장의 길이__)
- input_dim = 입력의 크기. (__자연어 처리에서는 단어 벡터의 차원__)


#### 은닉 상태 출력
***
- 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서

- 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서

    - RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정 가능
    - 마지막 은닉 상태만 전달하도록 하면 다 대 일(many-to-one) 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 RNN 은닉층이 하나 더 있는 경우이거나 다 대 다(many-to-many) 문제를 풀 수 있다.


#### Deep RNN
***
- 은닉층이 1개가 아닌 2개 이상 더 쌓은 모습
-------------
```python

model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

```
-------------

#### 양방향 RNN
***
- 양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라, 이후 시점의 입력 또한 예측에 기여할 수 있다.
- 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 할 수 있도록 고안된 것이 양방향 RNN

    - 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용.
    - 첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States) 를 전달받아 현재의 은닉 상태를 계산.
    - 두번째 메모리 셀은 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States) 를 전달 받아 현재의 은닉 상태를 계산. (__입력 시퀀스를 반대 방향으로 읽는 것__)

-------------
```python
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

```
--------------

In [2]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


2022-09-26 13:22:19.553861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


## Data

In [26]:
path = '../data/sample'

In [27]:
df = pd.read_csv(f'{path}/korean_correct_train_data_100000.csv')

In [28]:
df.head()

,src,tgt
0,증뉴수로 히서칸 탈지유와 수프는 생산자 지치메 표시된 농도의 절바느로 증뉴수로 만드럳따.,증류수로 희석한 탈지유와 수프는 생산자 지침에 표시된 농도의 절반으로 증류수로 만들었다.
1,다른 한펴느로는 파괴려게 대한 함니저기 뉴연성과 빌딩 블록꽈 신호 분자가 사라인는 ...,다른 한편으로는 파괴력에 대한 합리적인 유연성과 빌딩 블록과 신호 분자가 살아있는 ...
2,식푸뮈생감도근 기술과제임과 동시에 버베 따른 행정저 검무로서 감독짜는 견실하고 유연...,식품위생감독은 기술과제임과 동시에 법에 따른 행정적 업무로서 감독자는 견실하고 유연...
3,예시메 합껴칸 자는 시 품질감독뿌서가 성급 품질감독뿌서와 품질감독껌사거멱총구게 보고...,예심에 합격한 자는 시 품질감독부서가 성급 품질감독부서와 품질감독검사검역총국에 보고...
4,케이크 가루는 기포 분포가 매우 규닐하며 베이킹 후에도 월래 구조가 여전히 유지되고...,케이크 가루는 기포 분포가 매우 균일하며 베이킹 후에도 원래 구조가 여전히 유지되고...


In [29]:
x_train, x_test, y_train, y_test = train_test_split(df['src'], df['tgt'], test_size=0.2, shuffle=True, random_state=34)

In [30]:
y_train

48472    산양 거래시장과 농업정보시스템 건설을 강화하고 교통반경에 따라 점을 합리적으로 배치...
5937                   향후 연구에서는 멜라토닌과 같은 다른 약물도 테스트할 수 있다.
7467                    그러나 이 지역은 슈도모나스 종을 만족스럽게 구별하지 못한다.
2395     카파 경쇄의 체세포 돌연변이는 모체 항지랄레논 항체의 생체 내 친화성 성숙 동안 발...
68514           가공 과정과 생산 가공 과정이 공정 설계 요구사항을 충족하는지 여부가 있다.
                               ...                        
68693            일부 새로운 결과는 생물막 연구에 열량측정법을 적용할 수 있음을 보여준다.
93942    물품을 추적하기 위해 모든 포장 상자에는 과수원 등록 번호와 포장 공장 등록 번호를...
22377    사료 원료의 선택은 고품질 및 용이한 소화성을 원칙으로 하고 미코톡신이 함유된 사료...
43498                    방목지를 초원으로 되돌리는 프로젝트의 실행 속도를 높입니다.
77217    식초 양조 과정에는 많은 미생물이 관여하고 그에 따라 많은 대사 산물이 생성되지만 ...
Name: tgt, Length: 80000, dtype: object

In [31]:
x_train

48472    사냥 거래시장과 농업쩡보시스템 건서를 강화하고 교통반경에 따라 저믈 함니저그로 배치...
5937                   향후 연구에서는 멜라토닌과 가튼 다르 냥물도 테스트할 쑤 읻따.
7467                    그러나 이 지여근 슈도모나스 종을 만족쓰럽께 구별하지 모탄다.
2395     카파 경쇄의 체세포 도련벼니는 모체 항지랄레논 항체의 생체 내 친화성 성숙 똥안 발...
68514           가공 괓정과 생산 가공 과정이 공정 설계 요구사항을 충족하는지 여부가 있다.
                               ...                        
68693            일부 새로운 결과는 생물막 연구에 열량측정법을 적용핣 수 있음을 보여준다.
93942    물품을 추적하기 위해 모든 포장 짱자에는 과수원 등록 번호와 포장 공장 등록 번호를...
22377    사료 월료의 선태근 고품질 미 둉이한 소화성으 뤈치그로 하고 미코톡씨니 하뮤된 사료...
43498                    방목찌를 초워느로 되돌리는 프로젝트의 실행 속또를 로핌니다.
77217    식초 양조 과정에는 많은 미생물이 관여하고 그에 따라 많은 대사 산뭌이 생성되지만 ...
Name: src, Length: 80000, dtype: object

In [32]:
# vocabulary

src_words_counter = Counter([word for s in df['src'].tolist() for word in s.split()])
tgt_words_counter = Counter([word for s in df['tgt'].tolist() for word in s.split()])

In [33]:
src_words_counter

Counter({'증뉴수로': 14,
         '히서칸': 4,
         '탈지유와': 3,
         '수프는': 5,
         '생산자': 61,
         '지치메': 46,
         '표시된': 101,
         '농도의': 195,
         '절바느로': 6,
         '만드럳따.': 25,
         '다른': 3995,
         '한펴느로는': 18,
         '파괴려게': 1,
         '대한': 9311,
         '함니저기': 10,
         '뉴연성과': 2,
         '빌딩': 10,
         '블록꽈': 3,
         '신호': 114,
         '분자가': 61,
         '사라인는': 87,
         '세포에': 243,
         '드러갈': 30,
         '쑤': 9176,
         '읻또록': 138,
         '충분한': 189,
         '투과성으': 3,
         '류지해야': 25,
         '한다.': 3511,
         '식푸뮈생감도근': 1,
         '기술과제임과': 1,
         '동시에': 791,
         '버베': 176,
         '따른': 234,
         '행정저': 2,
         '검무로서': 1,
         '감독짜는': 2,
         '견실하고': 1,
         '유연한': 19,
         '기술지식': 1,
         '끼바늘': 17,
         '갇추어야': 39,
         '하며': 1047,
         '범뉴레': 52,
         '기초지식꽈': 1,
         '행정에': 2,
         '자겨글': 59,
         '갇춘': 67,
         '정치적': 32,
 

In [34]:
tgt_words_counter

Counter({'증류수로': 23,
         '희석한': 6,
         '탈지유와': 3,
         '수프는': 6,
         '생산자': 69,
         '지침에': 85,
         '표시된': 112,
         '농도의': 223,
         '절반으로': 9,
         '만들었다.': 51,
         '다른': 4760,
         '한편으로는': 42,
         '파괴력에': 1,
         '대한': 11185,
         '합리적인': 194,
         '유연성과': 6,
         '빌딩': 10,
         '블록과': 3,
         '신호': 125,
         '분자가': 66,
         '살아있는': 212,
         '세포에': 263,
         '들어갈': 74,
         '수': 18298,
         '있도록': 566,
         '충분한': 221,
         '투과성을': 30,
         '유지해야': 66,
         '한다.': 3838,
         '식품위생감독은': 1,
         '기술과제임과': 1,
         '동시에': 844,
         '법에': 356,
         '따른': 246,
         '행정적': 22,
         '업무로서': 1,
         '감독자는': 3,
         '견실하고': 1,
         '유연한': 24,
         '기술지식': 1,
         '기반을': 127,
         '갖추어야': 74,
         '하며': 1093,
         '법률에': 97,
         '기초지식과': 2,
         '행정에': 2,
         '자격을': 141,
         '갖춘': 143,
         '정치

In [35]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
 
    return x_tk.texts_to_sequences(x), x_tk

In [36]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')

## Preprocessing

In [37]:
def preprocess(x,y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
 
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
 
    return preprocess_x, preprocess_y, x_tk, y_tk

In [38]:
prep_x, prep_y, x_tok, y_tok = preprocess(df['src'], df['tgt'])

max_src_sequence_length = prep_x.shape[1]
max_tgt_sequence_length = prep_y.shape[1]
src_vocab_size = len(x_tok.word_index)
tgt_vocab_size = len(y_tok.word_index)

In [39]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
 
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [40]:
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dropout(0.5))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 
     
     
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [60]:
tmpx = pad(prep_x, max_tgt_sequence_length)
tmpx = tmpx.reshape((-1,prep_y.shape[-2], 1 ))

In [ ]:
simple_rnn_model = simple_model(
    tmpx.shape,
    max_tgt_sequence_length,
    src_vocab_size,
    tgt_vocab_size)

simple_rnn_model.fit(tmpx, prep_y, batch_size=300, epochs=10, validation_split=0.2)

Epoch 1/10


In [11]:
from keras.models import load_model

In [12]:
model = load_model('../rnn_model.h5')

2022-09-27 16:17:35.759005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 54, 64)            4224      
                                                                 
 dropout_9 (Dropout)         (None, 54, 64)            0         
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 54, 64)            8256      
                                                                 
 dropout_10 (Dropout)        (None, 54, 64)            0         
                                                                 
 dense_6 (Dense)             (None, 54, 128)           8320      
                                                                 
 dropout_11 (Dropout)        (None, 54, 128)           0         
                                                                 
 dense_7 (Dense)             (None, 54, 124786)       

In [48]:
a = model.predict(testx[0])

2/2 [==============================] - 0s 11ms/step


In [41]:
prep_test_x, prep_test_y, x_test_tok, y_test_tok = preprocess(x_test, y_test)

In [44]:
max_src_sequence_length = prep_test_x.shape[1]
max_tgt_sequence_length = prep_test_y.shape[1]

In [45]:
testx = pad(prep_test_x, max_tgt_sequence_length)
testx = testx.reshape((-1,prep_test_y.shape[-2], 1 ))

In [49]:
a

array([[[5.0765004e-05, 3.8329341e-05, 2.9963476e-05, ...,
         4.1886856e-06, 4.2055672e-06, 4.2795036e-06]],

       [[5.0764997e-05, 3.8329334e-05, 2.9963470e-05, ...,
         4.1886847e-06, 4.2055663e-06, 4.2795032e-06]],

       [[1.1627232e-03, 3.1457234e-03, 1.4542310e-03, ...,
         1.3791941e-09, 1.4529481e-09, 1.5616815e-09]],

       ...,

       [[1.2791483e-01, 1.1927619e-02, 1.0912936e-02, ...,
         6.8340718e-08, 6.9383574e-08, 7.4430041e-08]],

       [[1.2791483e-01, 1.1927619e-02, 1.0912936e-02, ...,
         6.8340718e-08, 6.9383574e-08, 7.4430041e-08]],

       [[1.2791483e-01, 1.1927619e-02, 1.0912936e-02, ...,
         6.8340718e-08, 6.9383574e-08, 7.4430041e-08]]], dtype=float32)